# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("../output_data/cities.csv", encoding="utf-8")
cities_df

,Unnamed: 0,City,Country,Latitude,Longitudes,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date
0,0,vaini,TO,-21.2000,-175.2000,80.60,78,40,9.22,1618792670
1,3,kapaa,US,22.0752,-159.3190,78.80,61,40,13.80,1618792093
2,4,provideniya,RU,64.3833,-173.3000,30.20,69,0,6.71,1618792333
3,5,egvekinot,RU,66.3167,-179.1667,27.79,98,100,12.46,1618792333
4,7,alofi,NU,-19.0595,-169.9187,84.20,54,5,10.36,1618792333
...,...,...,...,...,...,...,...,...,...,...
539,593,ngunguru,NZ,-35.6167,174.5000,70.00,65,71,5.01,1618792404
540,594,russell,NZ,32.3502,-85.1999,69.80,59,20,3.44,1618792405
541,595,asau,TV,46.4333,26.4000,41.00,67,100,1.25,1618792405
542,597,beringovskiy,RU,63.0500,179.3167,32.94,95,88,3.36,1618792405


In [3]:
cities_df[cities_df["Latitude"]<-90]

,Unnamed: 0,City,Country,Latitude,Longitudes,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date


In [4]:
# sam_locations = [(cities_df.iloc[i,3],cities_df.iloc[i,4]) for i in range(30,544,200)]
# sam_weights = [cities_df.iloc[i,6] for i in range(30,544,200)]
# max_humidity = cities_df['Humidity (%)'].max()
# print(sam_weights)
# print(max_humidity)
# sam_locations 


In [5]:
locations = cities_df[['Latitude','Longitudes']]
weights = cities_df['Humidity (%)'].astype(float)
max_humidity = cities_df['Humidity (%)'].max()
print(weights)
locations 

0      78.0
1      61.0
2      69.0
3      98.0
4      54.0
       ... 
539    65.0
540    59.0
541    67.0
542    95.0
543    65.0
Name: Humidity (%), Length: 544, dtype: float64


,Latitude,Longitudes
0,-21.2000,-175.2000
1,22.0752,-159.3190
2,64.3833,-173.3000
3,66.3167,-179.1667
4,-19.0595,-169.9187
...,...,...
539,-35.6167,174.5000
540,32.3502,-85.1999
541,46.4333,26.4000
542,63.0500,179.3167


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

# Customize the size of the figure
figure_layout = {
    'width':'900px',
    'height':'500px',
    'border':'1px solid black',
    'padding':'1px',
    'margin':'0 auto 0 auto'
}
fig = gmaps.figure(layout = figure_layout)
heat_layer = gmaps.heatmap_layer(locations,weights= weights,dissipating=False,max_intensity=max_humidity,point_radius=2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
ideal_weather_cities_df = cities_df.loc[(cities_df["Max Temperature (F)"]>70)& (cities_df["Max Temperature (F)"]<80)&(cities_df["Wind Speed (mph)"]<10)&(cities_df["Cloudiness (%)"]==0)].dropna()
ideal_weather_cities_df

,Unnamed: 0,City,Country,Latitude,Longitudes,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date
77,86,la libertad,SV,-2.2333,-80.9000,78.80,78,0,5.75,1618792131
129,140,presidencia roque saenz pena,AR,-26.7852,-60.4388,73.99,81,0,1.99,1618792349
150,162,cardoso,BR,-20.0819,-49.9142,70.99,65,0,6.93,1618792351
192,207,oussouye,SN,12.4850,-16.5469,75.20,83,0,4.61,1618792356
273,296,port alfred,ZA,-33.5906,26.8910,74.53,33,0,9.57,1618792367
274,297,east london,ZA,-33.0153,27.9116,74.48,29,0,7.14,1618792286
285,309,sohag,EG,26.5570,31.6948,78.80,29,0,5.75,1618792368
308,335,umm lajj,SA,25.0213,37.2685,79.12,62,0,4.85,1618792371
309,336,sakakah,SA,29.9697,40.2064,73.40,20,0,6.91,1618792371
340,374,salalah,OM,17.0151,54.0924,78.80,73,0,3.44,1618792160


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [39]:
hotel_df = ideal_weather_cities_df[["City","Country","Latitude","Longitudes"]]
hotel_df["Hotel Name"] = ""
hotel_df

<ipython-input-39-753a8f7add3c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Latitude,Longitudes,Hotel Name
77,la libertad,SV,-2.2333,-80.9000,
129,presidencia roque saenz pena,AR,-26.7852,-60.4388,
150,cardoso,BR,-20.0819,-49.9142,
192,oussouye,SN,12.4850,-16.5469,
273,port alfred,ZA,-33.5906,26.8910,
274,east london,ZA,-33.0153,27.9116,
285,sohag,EG,26.5570,31.6948,
308,umm lajj,SA,25.0213,37.2685,
309,sakakah,SA,29.9697,40.2064,
340,salalah,OM,17.0151,54.0924,


In [40]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "radius" : 5000,
          "key" : g_key}

for index,row in hotel_df.iterrows():
    city = row["City"]
    lat = row["Latitude"]
    lng = row["Longitudes"]
    
    params["location"] = f"{lat},{lng}"
    response = requests.get(base_url, params=params).json()
    hotel_name = response['results'][0]['name']
    hotel_df.loc[index,"Hotel Name"] = hotel_name
    print(f"The nearest hotel to {city} is {hotel_name}")
    
hotel_df

C:\Users\John\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


The nearest hotel to la libertad is La Libertad
The nearest hotel to presidencia roque saenz pena is Presidencia Roque Sáenz Peña
The nearest hotel to cardoso is Cardoso
The nearest hotel to oussouye is Oussouye
The nearest hotel to port alfred is Port Alfred
The nearest hotel to east london is East London
The nearest hotel to sohag is Sohag
The nearest hotel to umm lajj is Umluj
The nearest hotel to sakakah is Sakaka
The nearest hotel to salalah is Salalah
The nearest hotel to port hedland is Port Hedland
The nearest hotel to biloela is Biloela
The nearest hotel to port macquarie is Port Macquarie


,City,Country,Latitude,Longitudes,Hotel Name
77,la libertad,SV,-2.2333,-80.9000,La Libertad
129,presidencia roque saenz pena,AR,-26.7852,-60.4388,Presidencia Roque Sáenz Peña
150,cardoso,BR,-20.0819,-49.9142,Cardoso
192,oussouye,SN,12.4850,-16.5469,Oussouye
273,port alfred,ZA,-33.5906,26.8910,Port Alfred
274,east london,ZA,-33.0153,27.9116,East London
285,sohag,EG,26.5570,31.6948,Sohag
308,umm lajj,SA,25.0213,37.2685,Umluj
309,sakakah,SA,29.9697,40.2064,Sakaka
340,salalah,OM,17.0151,54.0924,Salalah


In [41]:
hotel_df = hotel_df.rename(columns ={"Latitude":"Lat","Longitudes":"Lng"})
hotel_df

,City,Country,Lat,Lng,Hotel Name
77,la libertad,SV,-2.2333,-80.9000,La Libertad
129,presidencia roque saenz pena,AR,-26.7852,-60.4388,Presidencia Roque Sáenz Peña
150,cardoso,BR,-20.0819,-49.9142,Cardoso
192,oussouye,SN,12.4850,-16.5469,Oussouye
273,port alfred,ZA,-33.5906,26.8910,Port Alfred
274,east london,ZA,-33.0153,27.9116,East London
285,sohag,EG,26.5570,31.6948,Sohag
308,umm lajj,SA,25.0213,37.2685,Umluj
309,sakakah,SA,29.9697,40.2064,Sakaka
340,salalah,OM,17.0151,54.0924,Salalah


In [47]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [48]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…